# Neural Network - Days to Death (DDT)

This ML model predicts days from admission to death for the patients in the MIMIC dataset who expired

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense

Using TensorFlow backend.


# Data Setup
Creates separate clean dataframes for both sets of patients - lived & died

In [22]:
labs = pd.read_csv('../Resources/labsNew.csv')
labs.head()

,SUBJECT_ID,HADM_ID,Albumin,Anion Gap,Bicarbonate,"Bilirubin, Total",Chloride,"Chloride, Whole Blood",Creatinine,Glucose_Blood_Gas,...,PT,PTT,Phosphate,Platelet Count,Potassium,"Potassium, Whole Blood",Sodium,"Sodium, Whole Blood",Urea Nitrogen,White Blood Cells
0,2,163353.0,0.0,0.0,0.0,9.3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.1
1,3,145834.0,1.8,17.0,25.0,0.8,99.0,114.0,3.2,265.0,...,14.8,125.7,4.8,179.0,5.4,3.7,136.0,139.0,36.0,15.1
2,4,185777.0,2.8,17.0,24.0,2.2,97.0,0.0,0.5,0.0,...,12.3,31.3,3.2,207.0,3.1,0.0,135.0,0.0,9.0,9.7
3,5,178980.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,309.0,0.0,0.0,0.0,0.0,0.0,13.9
4,6,107064.0,2.7,17.0,16.0,0.2,107.0,95.0,3.5,106.0,...,12.5,55.2,4.1,198.0,4.9,4.2,135.0,135.0,86.0,22.7


In [23]:
died = pd.read_csv('../Resources/admissions_died.csv')
died.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DAYS_TO_DEATH
0,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1,7.647917
1,55,56,181711,2104-01-02 02:01:00,2104-01-08 10:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,NOT SPECIFIED,NaN,WHITE,2104-01-01 23:59:00,2104-01-02 03:33:00,HEAD BLEED,1,1,6.353472
2,61,61,189535,2119-01-04 18:12:00,2119-02-03 01:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Private,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,1,1,29.307639
3,68,67,155252,2157-12-02 00:45:00,2157-12-02 03:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,JEWISH,SINGLE,WHITE,2157-12-01 20:45:00,2157-12-02 00:55:00,SUBARACHNOID HEMORRHAGE,1,1,0.131944
4,86,84,166401,2196-04-14 04:02:00,2196-04-17 13:42:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,NaN,OTHER,MARRIED,WHITE,2196-04-13 22:23:00,2196-04-14 04:31:00,"GLIOBLASTOMA,NAUSEA",1,1,3.402778


In [27]:
died_df = pd.merge(died, labs, on='HADM_ID')
died_df.head()

,ROW_ID,SUBJECT_ID_x,HADM_ID,ADMITTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,PT,PTT,Phosphate,Platelet Count,Potassium,"Potassium, Whole Blood",Sodium,"Sodium, Whole Blood",Urea Nitrogen,White Blood Cells
0,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,...,13.2,29.4,2.7,109.0,3.3,4.2,128.0,0.0,13.0,6.9
1,55,56,181711,2104-01-02 02:01:00,2104-01-08 10:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,...,13.0,27.0,3.9,210.0,4.0,3.4,128.0,138.0,21.0,10.0
2,61,61,189535,2119-01-04 18:12:00,2119-02-03 01:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Private,NaN,...,11.1,28.7,2.7,21.0,3.3,0.0,139.0,0.0,17.0,0.1
3,68,67,155252,2157-12-02 00:45:00,2157-12-02 03:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,...,13.2,35.5,0.0,183.0,5.1,0.0,137.0,0.0,24.0,9.3
4,86,84,166401,2196-04-14 04:02:00,2196-04-17 13:42:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,NaN,...,0.0,0.0,3.4,231.0,2.5,0.0,127.0,0.0,6.0,11.6


In [28]:
died_df.columns

Index(['ROW_ID', 'SUBJECT_ID_x', 'HADM_ID', 'ADMITTIME', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION',
       'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY',
       'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'DAYS_TO_DEATH', 'SUBJECT_ID_y', 'Albumin',
       'Anion Gap', 'Bicarbonate', 'Bilirubin, Total', 'Chloride',
       'Chloride, Whole Blood', 'Creatinine', 'Glucose_Blood_Gas',
       'Glucose_Chemistry', 'Hematocrit', 'Hematocrit, Calculated',
       'Hemoglobin_Blood_Gas', 'Hemoglobin_Hematology', 'INR(PT)', 'Lactate',
       'Magnesium', 'PT', 'PTT', 'Phosphate', 'Platelet Count', 'Potassium',
       'Potassium, Whole Blood', 'Sodium', 'Sodium, Whole Blood',
       'Urea Nitrogen', 'White Blood Cells'],
      dtype='object')

In [30]:
died_df = died_df.drop(['ROW_ID', 'SUBJECT_ID_x', 'SUBJECT_ID_y', 'HADM_ID', 'ADMITTIME', 'DEATHTIME', 'DISCHARGE_LOCATION',
                         'RELIGION', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'HOSPITAL_EXPIRE_FLAG',
       'HAS_CHARTEVENTS_DATA', 'LANGUAGE'], axis = 1)

In [32]:
died_df_dummy = pd.get_dummies(died_df)
died_df_dummy.head()

,DAYS_TO_DEATH,Albumin,Anion Gap,Bicarbonate,"Bilirubin, Total",Chloride,"Chloride, Whole Blood",Creatinine,Glucose_Blood_Gas,Glucose_Chemistry,...,DIAGNOSIS_VT WITH AICD FITTING,DIAGNOSIS_VTACH;CONGESTIVE HEART FAILURE;ACUTE RENAL FAILURE;COLITIS,DIAGNOSIS_WEAKNESS,"DIAGNOSIS_WEAKNESS, DIARRHEA",DIAGNOSIS_WEAKNESS;C-DIFF DIARRHEA,DIAGNOSIS_WEAKNESS;DEHYDRATION,DIAGNOSIS_WEBSTER TRIPLE A,DIAGNOSIS_WIDE COMPLEX TACHYCARDIA,DIAGNOSIS_WORSEING RENAL FAILURE,DIAGNOSIS_WOUND INFECTION
0,7.647917,2.7,9.0,27.0,0.9,95.0,0.0,0.9,116.0,110.0,...,0,0,0,0,0,0,0,0,0,0
1,6.353472,3.2,14.0,27.0,0.0,91.0,0.0,0.7,0.0,155.0,...,0,0,0,0,0,0,0,0,0,0
2,29.307639,3.0,9.0,26.0,0.6,107.0,0.0,0.3,0.0,98.0,...,0,0,0,0,0,0,0,0,0,0
3,0.131944,0.0,22.0,20.0,0.0,100.0,0.0,1.6,0.0,176.0,...,0,0,0,0,0,0,0,0,0,0
4,3.402778,0.0,14.0,22.0,0.3,94.0,0.0,0.4,0.0,119.0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# assignment and reshaping of DIED data
data = died_df_dummy.drop(["DAYS_TO_DEATH"], axis=1)
target = died_df_dummy["DAYS_TO_DEATH"].values.reshape(-1, 1)

print(f"data shape: {data.shape}")
print(f"target shape: {target.shape}")

data shape: (5818, 31)
target shape: (5818, 1)


# Length of Stay Model

# Days to Death Model